<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/%EC%8A%B9%ED%98%9C/img_preprocessing_def(%EC%B5%9C%EC%A2%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import 

import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import glob
import math
# import image_dehazer

# contrast 함수 

In [ ]:
# contrast 함수
def contrast_roi(img, low, high):
    h, w = img.shape
    img_ = np.zeros(img.shape, dtype=np.uint8)
    for y in range(h):
        for x in range(w):
            temp = int((255 / (high - low)) * (img[y][x] - low))
            if temp > 255:
                img_[y][x] = 255
            elif temp < 0:
                img_[y][x] = 0
            else:
                img_[y][x] = temp
    return img_

# brightness 함수

In [ ]:
  def bright_ness(img):
    cols, rows = img.shape[:2]
    brightness = np.sum(img) / (255 * cols * rows)
    return brightness

## total 함수

In [ ]:
# 이미지 1개 가져오기
from google.colab import files
uploaded = files.upload()

In [ ]:
# 이미지 1개 img 적용
img = cv2.imread('166_F.jpg',cv2.IMREAD_COLOR)

In [ ]:
# 이미지 전처리 함수 (img, 모폴로지값1, 모폴로지값2, 필터값(d,e))
def merong(img,a,b,d,e):
  # 이미지 가져오기
  # img = cv2.imread('1_F_손목 - 복사본.jpg',cv2.IMREAD_COLOR)
  img1 = img

  # 밝기 조정
  if bright_ness(img1) > 0.8:
    img1 = np.clip(img1 - 80., 0, 255).astype(np.uint8)
  elif bright_ness(img1) > 0.75:
    img1 = np.clip(img1 - 50., 0, 255).astype(np.uint8)
  elif bright_ness(img1) > 0.65:
    img1 = np.clip(img1 - 30., 0, 255).astype(np.uint8)
  else: img1 = np.clip(img1 - 10., 0, 255).astype(np.uint8)

  # plt.imshow(img1)
  # plt.show()
  
  # dehazing (must be a color image)
  # img1 = image_dehazer.remove_haze(img1)		# Remove Haze

  # 이미지 Lab
  img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2BGR)
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2Lab)

  # 모폴리지
  k = cv2.getStructuringElement(cv2.MORPH_CROSS, (a, a))
  img1 = cv2.morphologyEx(img1, cv2.MORPH_TOPHAT, k) # 밝기 값이 크게 튀는 곳을 강조

  # 필터
  img1 = cv2.bilateralFilter(img1,-1, d, e)
  # img1= guided_Fitler(img1, img1, r=3, eps=100)

  # Lab to gray for binary
  img1 = cv2.cvtColor(img1, cv2.COLOR_Lab2BGR)
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

  # 정규화
  img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX)

  # 평탄화 (Clahe)
  img1 = cv2.equalizeHist(img1)
  clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(3,3)) #CLAHE 생성
  img1= clahe.apply(img1)           #CLAHE 적용


  # binary
  ret, mask = cv2.threshold(img1,
                            np.mean(img1), # thresh= 이미지의 평균값을 임계치로
                            255,
                            cv2.THRESH_BINARY) # ret - 임계값, mask = img마스크
  
  # 같은 픽셀 강도를 갖는 애들 연결해줌 (오브젝트 검출)
  contours, hierarchy = cv2.findContours(mask, # 마스크에서
                                         cv2.RETR_EXTERNAL, # 바깥쪽 라인만
                                         cv2.CHAIN_APPROX_SIMPLE) # 컨투어 꼭짓점 좌표만 제공 

  cv2.drawContours(mask, contours, -1, (255,255,255), -1) # -1: 모든 컨트어 표시 /color/ fill


  # plt.imshow(mask)
  # plt.show()










  #### 강 조
  img2 = img.copy()
  if bright_ness(img2) > 0.8:
    img2 = np.clip(img2 - 80., 0, 255).astype(np.uint8)
  elif bright_ness(img2) > 0.75:
    img2 = np.clip(img2 - 50., 0, 255).astype(np.uint8)
  elif bright_ness(img2) > 0.65:
    img2 = np.clip(img2 - 30., 0, 255).astype(np.uint8)
  else: img2 = np.clip(img2 - 10., 0, 255).astype(np.uint8)


  # 모폴로지
  k2 = cv2.getStructuringElement(cv2.MORPH_CROSS,(b,b))
  img2 = cv2.morphologyEx(img2, cv2.MORPH_TOPHAT, k2)

  # contrast
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  # img2= contrast(img2,mask,c)
  # if img2.mean() <= 15:
  #     low = img2.mean() * 2.3
  #     high = img2.mean() * 2.5
  # elif img2.mean() <= 20:
  #     low = img2.mean() * 2.3
  #     high = img2.mean() * 2.8
  # else:
  #     low = img2.mean() * 2.3
  #     high = img2.mean() * 3
  if img2.mean() <= 15:
      low = img2.mean() * 3
      high = img2.mean() * 3.5
  elif img2.mean() <= 20:
      low = img2.mean() * 3
      high = img2.mean() * 3.6
  else:
      low = img2.mean() * 3
      high = img2.mean() * 3.7

  img2 = cv2.blur(img2,(2,2))
  img2 = contrast_roi(img2, low, high)



    
  # # 뼈강조하고 마스크랑 비트 연산
  # # 모폴로지
  # k2 = cv2.getStructuringElement(cv2.MORPH_CROSS, (c, c))
  # img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k2)


  # 컨투어
  contours, hierarchy = cv2.findContours(img2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(img2, contours, -1, (255, 255, 255), -1)

  # 마스크랑 비트 연산 - 이미지랑 마스크 둘 다에서 흰색인 부분만 출력
  img2 = cv2.bitwise_and(img2, mask) 
  # img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
  img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
  img2 = cv2.blur(img2,(2,2))

  plt.figure(figsize=(12,10))
  plt.imshow(img2,'gray')
  plt.show()

# merong(img,60,55,25,10)


# path설정으로 폴더 사진 전체에 적용 

In [ ]:
# 이미지 여러개에 함수 적용
import os
path = '/content/drive/MyDrive/cakd_colab/2차플젝/데이터 원본/데이터 원본/image/Female'
dir_list = os.listdir(path)
print(dir_list)

# 빼고싶은 개수만큼 [:x] 로 적어주세요
for item in dir_list[:20] :   
  img = cv2.imread(path+'/'+f'{item}',cv2.IMREAD_COLOR)
  # merong(img,55,25,25,10) # 원래꺼 (스킨은 잘 안빠지고 지골이 잘 빠짐)
  # merong(img,55,60,25,10)  # 새로운거 (지골이 잘 안빠지고 스킨이 잘 빠짐)
  merong(img,60,55,25,10) # 이건 다 잘되지롱
